In [1]:
addprocs(CPU_CORES)

@everywhere begin
  using StochasticDiffEq, DiffEqProblemLibrary, ParallelDataTransfer
  srand(99 + myid())
  prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob.tspan = (0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,RKMil(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRIW1(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRI(),dt=1/2^(18),adaptive=false,save_timeseries=false)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),dt=1/2^(18),adaptive=false,save_timeseries=false)
  Int(sol.u[1]!=NaN)
  println("Compilation complete.")
  js = 16:21
  dts = 1./2.^(js)
  fails = Array{Int}(length(dts),3)
  times = Array{Float64}(length(dts),3)
  numRuns = 10000
end
println("Setup Complete")

  likely near In[1]:1
 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
) in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
) in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10WARNING: Method definition .
#solve(Array{Any, 1}WARNING: Method definition , DiffEqBase#

	From worker 13:	Solve once to compile.
	From worker 7:	Solve once to compile.
	From worker 10:	Solve once to compile.
	From worker 8:	Solve once to compile.
	From worker 14:	Solve once to compile.
	From worker 2:	Solve once to compile.
	From worker 11:	Solve once to compile.
	From worker 9:	Solve once to compile.
	From worker 4:	Solve once to compile.
	From worker 3:	Solve once to compile.
	From worker 5:	Solve once to compile.
	From worker 17:	Solve once to compile.
	From worker 15:	Solve once to compile.
	From worker 12:	Solve once to compile.
	From worker 16:	Solve once to compile.
	From worker 6:	Solve once to compile.
	From worker 13:	Compilation complete.
	From worker 14:	Compilation complete.
	From worker 11:	Compilation complete.
	From worker 7:	Compilation complete.
	From worker 4:	Compilation complete.
	From worker 8:	Compilation complete.
	From worker 15:	Compilation complete.
	From worker 2:	Compilation complete.
	From worker 9:	Compilation complete.
	From worker 10:	Compi

, Array{Expr, 1}, Array{Symbol, 1}, Array{SymEngine.Basic, 1}, ArrayWARNING: Method definition {SymEngine(::.BasicType, {2}ParameterizedFunctions, .ArrayLotkaVolterra{}SymEngine).(ExprBasic, , 2Array}{, Expr, Array1{}SymEngine, .ArrayBasic{, SymEngine2.}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Array{SymEngine.Basic, 2}, Expr, Expr, Expr, Expr, ExprWARNING: Method definition , (::ExprType{, ParameterizedFunctionsExpr., LotkaVolterraExpr}), (ExprAny, , ExprAny, , AnyExpr, Any, , ArrayAny{, SymbolAny, , Any1, }Any, , AnyFloat64, , AnyFloat64, , Any, Float64Any), Any in module ParameterizedFunctions, Any at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25, Any overwritten,  at /home/crackauc/.julia/v0.5/ParameterizedFunctions/src/maketype.jl:25Any, .
Any, Any, Any, Any, Any, Any, Any, Any, Any, Any, Any, Any, Any, Any) in module ParameterizedFunctions at /home/crackauc/.julia/v0.5/ParameterizedFunction

In [8]:
## Timing Runs

@everywhere function runAdaptiveSRIW1(i)
  sol = solve(prob,SRIW1(),abstol=2.0^(-20),reltol=2.0^(-14),maxIters=Int(1e10),qmax=1.125,save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIW1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 41.74611805


In [11]:
## Timing Runs

@everywhere function runAdaptiveSRI(i)
  sol = solve(prob,SRI(),abstol=2.0^(-20),reltol=2.0^(-14),maxIters=Int(1e10),qmax=1.125,save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRI,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

.


The number of Adaptive Fails is 0. Elapsed time was 183.162530958


In [3]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt1(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),abstol=2.0^(-15),reltol=2.0^(-11),maxIters=Int(1e10),qmax=1.125,save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 28.849052188


In [5]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt2(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt2()),abstol=2.0^(-18),reltol=2.0^(-15),maxIters=Int(1e10),qmax=1.125,save_timeseries=false,verbose=false)
    Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt2,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 7. Elapsed time was 183.455265254


In [6]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt3(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt3()),abstol=2.0^(-16),reltol=2.0^(-14),maxIters=Int(1e10),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt3,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 77.176634881


In [7]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt4(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt4()),abstol=2.0^(-14),reltol=2.0^(-10),maxIters=Int(1e10),qmax=1.125,save_timeseries=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt4,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 24.182544955


In [8]:
@everywhere function runEM(i,j)
  sol =solve(prob,EM(),dt=dts[j],maxIters=Int(1e11),save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t1 = @elapsed numFails = sum(pmap((i)->runEM(i,j),1:numRuns))
  println("The number of Euler-Maruyama Fails is $numFails. Elapsed time was $t1")
  fails[j,1] = numFails
  times[j,1] = t1
end

j = 1
The number of Euler-Maruyama Fails is 2265. Elapsed time was 41.73240955
j = 2
The number of Euler-Maruyama Fails is 829. Elapsed time was 96.302097862
j = 3
The number of Euler-Maruyama Fails is 953. Elapsed time was 184.146256885
j = 4
The number of Euler-Maruyama Fails is 427. Elapsed time was 391.201677899
j = 5
The number of Euler-Maruyama Fails is 813. Elapsed time was 745.987267463
j = 6
The number of Euler-Maruyama Fails is 296. Elapsed time was 1563.576991868


In [12]:
@everywhere function runSRI(i,j)
  sol =solve(prob,SRIW1(),dt=dts[j],maxIters=Int(1e11),adaptive=false,save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in 1:4
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t2 = @elapsed numFails = sum(pmap((i)->runSRI(i,j),1:numRuns))
  println("The number of Rossler-SRI Fails is $numFails. Elapsed time was $t2")
  fails[j,2] = numFails
  times[j,2] = t2
end

j = 1
The number of Rossler-SRI Fails is 94. Elapsed time was 69.143927077
j = 2
The number of Rossler-SRI Fails is 15. Elapsed time was 136.101493517
j = 3
The number of Rossler-SRI Fails is 4. Elapsed time was 271.535782867
j = 4
The number of Rossler-SRI Fails is 0. Elapsed time was 535.333766774


In [13]:
@everywhere function runMil(i,j)
  sol =solve(prob,RKMil(),dt=dts[j],maxIters=Int(1e11),save_timeseries=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t3 = @elapsed numFails = sum(pmap((i)->runMil(i,j),1:numRuns))
  println("The number of RK-Milstein Fails is $numFails. Elapsed time was $t3")
  fails[j,3] = numFails
  times[j,3] = t3
end

j = 1
The number of RK-Milstein Fails is 3749. Elapsed time was 32.604962011
j = 2
The number of RK-Milstein Fails is 2424. Elapsed time was 76.770625324
j = 3
The number of RK-Milstein Fails is 3033. Elapsed time was 140.84485195
j = 4
The number of RK-Milstein Fails is 3962. Elapsed time was 245.852010095
j = 5
The number of RK-Milstein Fails is 2649. Elapsed time was 594.204890017
j = 6
The number of RK-Milstein Fails is 2236. Elapsed time was 1244.17235086


In [ ]:
using Plots
lw = 3
p2 = plot(dts,times,xscale=:log2,yscale=:log2,guidefont=font(16),tickfont=font(14),yguide="Elapsed Time (s)",xguide=L"Chosen $\Delta t$",top_margin=50px,linewidth=lw,lab=["Euler-Maruyama" "RK-Mil" "RosslerSRI"],legendfont=font(14))
plot!(dts,repmat([adaptiveTime],11),linewidth=lw,line=:dash,lab="ESRK+RSwM3",left_margin=75px)
scatter!([2.0^(-20);2.0^(-20);2.0^(-18)],[times[5,1];times[5,2];times[3,3]],markersize=20,c=:red,lab="")
plot(p2,size=(800,800))